<a href="https://colab.research.google.com/github/BenWilop/MechInt/blob/main/colabs/intro/Intro_to_Weights_%26_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_&_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab} -->

<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_&_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{intro-colab} -->

Use [W&B](https://wandb.ai/site?utm_source=intro_colab&utm_medium=code&utm_campaign=intro) for machine learning experiment tracking, model checkpointing, collaboration with your team and more. See the full W&B Documentation [here](https://docs.wandb.ai/).

In this notebook, you will create and track a machine learning experiment using a simple PyTorch model. By the end of the notebook, you will have an interactive project dashboard that you can share and customize with other members of your team. [View an example dashboard here](https://wandb.ai/wandb/wandb_example).

## Prerequisites

Install the W&B Python SDK and log in:

In [32]:
!pip install wandb -qU

In [33]:
# Log in to your W&B account
import wandb
import random
import math

In [34]:
wandb.login()

True

In [35]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M")
print(timestamp)

2025-02-22-18-59


## Simulate and track a machine learning experiment with W&B

Create, track, and visualize a machine learning experiment. To do this:

1. Initialize a [W&B run](https://docs.wandb.ai/guides/runs) and pass in the hyperparameters you want to track.
2. Within your training loop, log metrics such as the accuruacy and loss.

In [38]:
import random
import math

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 1️. Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project=f"wanbtest_simulated_{timestamp}",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 2️. Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

acc,▁▆▆▆███▇
loss,█▅▂▁▁▂▁▂
acc,0.7065
loss,0.27912


acc,▁▃▂▇▄█▇█
loss,█▅▃▃▂▃▁▁
acc,0.95004
loss,0.04206


acc,▁▂▄▇▆▅█▆
loss,█▃▄▃▃▁▂▂
acc,0.79719
loss,0.19569


acc,▁▅▆▆▇▇██
loss,█▃▅▂▁▂▂▁
acc,0.89481
loss,0.08245


acc,▁▃▆▇▆▇▇█
loss,█▅▁▂▂▁▁▁
acc,0.87032
loss,0.20242


In [39]:
# raise Exception()

View how your machine learning peformed in your W&B project. Copy and paste the URL link that is printed from the previous cell. The URL will redirect you to a W&B project that contains a dashboard showing graphs the show how

The following image shows what a dashboard can look like:

![](https://i.imgur.com/Pell4Oo.png)

Now that we know how to integrate W&B into a psuedo machine learning training loop, let's track a machine learning experiment using a basic PyTorch neural network. The following code will also upload model checkpoints to W&B that you can then share with other teams in in your organization.

##  Track a machine learning experiment using Pytorch

The following code cell defines and trains a simple MNIST classifier. During training, you will see W&B prints out URLs. Click on the project page link to see your results stream in live to a W&B project.

W&B runs automatically log [metrics](https://docs.wandb.ai/ref/app/pages/run-page#charts-tab),
[system information](https://docs.wandb.ai/ref/app/pages/run-page#system-tab),
[hyperparameters](https://docs.wandb.ai/ref/app/pages/run-page#overview-tab),
[terminal output](https://docs.wandb.ai/ref/app/pages/run-page#logs-tab) and
you'll see an [interactive table](https://docs.wandb.ai/guides/data-vis)
with model inputs and outputs.

### Set up PyTorch Dataloader
The following cell defines some useful functions that we will need to train our machine learning model. The functions themselves are not unique to W&B so we'll not cover them in detail here. See the PyTorch documentation for more information on how to define [forward and backward training loop](https://pytorch.org/tutorials/beginner/nn_tutorial.html), how to use [PyTorch DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) to load data in for training, and how define PyTorch models using the [`torch.nn.Sequential` Class](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html).

In [40]:
#@title
import torch, torchvision
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as T

MNIST.mirrors = [mirror for mirror in MNIST.mirrors if "http://yann.lecun.com/" not in mirror]

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

### Create a teble to compare the predicted values versus the true value

The following cell is unique to W&B, so let's go over it.

In the cell we define a function called `log_image_table`. Though technically, optional, this function creates a W&B Table object. We will use the table object to create a table that shows what the model predicted for each image.

More specifically, each row will conists of the image fed to the model, along with predicted value and the actual value (label).

In [41]:
def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

### Train your model and upload checkpoints

The following code trains and saves model checkpoints to your project. Use model checkpoints like you normally would to assess how the model performed during training.

W&B also makes it easy to share your saved models and model checkpoints with other members of your team or organization. To learn how to share your model and model checkpoints with members outside of your team, see [W&B Registry](https://docs.wandb.ai/guides/registry).

In [43]:
import os
from datetime import datetime
# Launch 3 experiments, trying different dropout rates
for run in range(3):
    # Create a timestamp string in the format YYYYMMDD_HHMM
    dropout = random.uniform(0.01, 0.80)
    wandb.init(
        project=f"wanbtest_mnist_{timestamp}",
        name=f"dropout-{round(dropout, 4)}",
        config={
            "epochs": 5,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": dropout,
            })

    # Copy your config
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})

        # Save the model checkpoint to wandb
        torch.save(model, "my_model.pt")
        wandb.log_model("./my_model.pt", "my_mnist_model", aliases=[f"epoch-{epoch+1}_dropout-{round(wandb.config.dropout, 4)}"])

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # Close your wandb run
    wandb.finish()

Epoch: 1, Train Loss: 0.380, Valid Loss: 0.288890, Accuracy: 0.92
Epoch: 2, Train Loss: 0.197, Valid Loss: 0.240448, Accuracy: 0.93
Epoch: 3, Train Loss: 0.255, Valid Loss: 0.207029, Accuracy: 0.94
Epoch: 4, Train Loss: 0.200, Valid Loss: 0.184544, Accuracy: 0.94
Epoch: 5, Train Loss: 0.178, Valid Loss: 0.172041, Accuracy: 0.95


train/epoch,▁▁▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
train/train_loss,█▇▇▇▆▅▅▄▅▃▄▃▅▂▃▂▂▂▂▃▂▂▃▂▂▂▁▂▂▁▂▁▂▁▁▁▂▂▂▁
val/val_accuracy,▁▄▅▇█
val/val_loss,█▅▃▂▁
test_accuracy,0.8
train/epoch,5
train/example_ct,60000
train/train_loss,0.17811
val/val_accuracy,0.947
val/val_loss,0.17204


Epoch: 1, Train Loss: 0.500, Valid Loss: 0.356281, Accuracy: 0.91
Epoch: 2, Train Loss: 0.497, Valid Loss: 0.287657, Accuracy: 0.92
Epoch: 3, Train Loss: 0.373, Valid Loss: 0.261494, Accuracy: 0.92
Epoch: 4, Train Loss: 0.296, Valid Loss: 0.245664, Accuracy: 0.92
Epoch: 5, Train Loss: 0.317, Valid Loss: 0.229252, Accuracy: 0.93


train/epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/example_ct,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇███
train/train_loss,█▇▆▃▃▂▃▂▂▂▂▂▁▁▁▁▂▂▁▂▁▁▂▂▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁
val/val_accuracy,▁▆▅▇█
val/val_loss,█▄▃▂▁
test_accuracy,0.8
train/epoch,5
train/example_ct,60000
train/train_loss,0.31652
val/val_accuracy,0.927
val/val_loss,0.22925


Epoch: 1, Train Loss: 0.511, Valid Loss: 0.342930, Accuracy: 0.90
Epoch: 2, Train Loss: 0.238, Valid Loss: 0.276761, Accuracy: 0.92
Epoch: 3, Train Loss: 0.342, Valid Loss: 0.251735, Accuracy: 0.92
Epoch: 4, Train Loss: 0.257, Valid Loss: 0.235603, Accuracy: 0.93
Epoch: 5, Train Loss: 0.360, Valid Loss: 0.229118, Accuracy: 0.93


train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/train_loss,█▆▆▄▄▂▂▂▂▂▂▂▁▂▁▂▂▂▂▂▂▁▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
val/val_accuracy,▁▅▇▇█
val/val_loss,█▄▂▁▁
test_accuracy,0.8
train/epoch,5
train/example_ct,60000
train/train_loss,0.36009
val/val_accuracy,0.9295
val/val_loss,0.22912


# Load model

In [45]:
api = wandb.Api()

# Replace with the exact path to your artifact and alias
artifact = api.artifact("benwilop-rwth-aachen-university/wanbtest_mnist_2025-02-22-18-59/my_mnist_model:v9", type="model")

# Download the artifact to a local directory
artifact_dir = artifact.download()

# Load the checkpoint
model_path = os.path.join(artifact_dir, "my_model.pt")
model = torch.load(model_path)

wandb:   1 of 1 files downloaded.  
<ipython-input-45-088fd0387465>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [47]:
valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
loss_func = nn.CrossEntropyLoss()
validate_model(model, valid_dl, loss_func, log_images=False)

(tensor(0.2293, device='cuda:0'), 0.927)